In [1]:
import pandas as pd
import altair as alt
from google.cloud import bigquery
import geopandas as gpd
from shapely import wkt
from census import Census
from us import states
#import google.auth
#from google.cloud import bigquery
#from google.cloud import bigquery_storage
bqclient = bigquery.Client.from_service_account_json('C:/Users/bennd/Documents/MUSA509/TransitPolicyApp-99838a65a6ed.json')
pd.set_option('mode.chained_assignment', None)
#ridership = pd.read_csv("data/ridership.csv")
#ridership['month'] = pd.to_datetime(ridership['month'])
covid = pd.read_csv("data/us-counties.csv")
covid['date'] = pd.to_datetime(covid['date'])
top50 = pd.read_csv("data/top50.csv")

In [2]:
import plotly.express as px

In [3]:
covid_l = covid[covid["date"] == "2020-09-30"]
covid_l.head()

date   county    state  fips  cases  deaths
577661 2020-09-30  Autauga  Alabama  1001   1791    27.0
577662 2020-09-30  Baldwin  Alabama  1003   5640    52.0
577663 2020-09-30  Barbour  Alabama  1005    896     7.0
577664 2020-09-30     Bibb  Alabama  1007    664    11.0
577665 2020-09-30   Blount  Alabama  1009   1629    15.0

In [19]:
#Pull ridership data from Google BigQuery

query = f"""
    SELECT metro_area, area_land, area_water, GEOID10, mode, month, trips, classification, lat, lon
    FROM `transitpolicyapp.ridership.Ridership`
    GROUP BY metro_area, area_land, area_water, GEOID10, mode, month, trips, classification, lat, lon
"""
ridership = bqclient.query(query).to_dataframe()
ridership.head()

metro_area  area_land  area_water  GEOID10 mode    month  \
0  Monessen-California, PA  115663317     3688078    58168  Bus  2019-01   
1  Monessen-California, PA  115663317     3688078    58168  Bus  2019-02   
2  Monessen-California, PA  115663317     3688078    58168  Bus  2019-03   
3  Monessen-California, PA  115663317     3688078    58168  Bus  2019-04   
4  Monessen-California, PA  115663317     3688078    58168  Bus  2019-05   

   trips classification        lat        lon  
0  22447       Non-Rail  40.136158 -79.884233  
1  23338       Non-Rail  40.136158 -79.884233  
2  24577       Non-Rail  40.136158 -79.884233  
3  24723       Non-Rail  40.136158 -79.884233  
4  23115       Non-Rail  40.136158 -79.884233

In [5]:
#Generate interactive altair ridership chart by mode and month

search = "Philadelphia, PA-NJ-DE-MD"

ridership_s = ridership[ridership["metro_area"] == search]
chart_ridership = alt.Chart(ridership_s).mark_area().encode(
    x=alt.X('month:T', axis=alt.Axis(title='Month')),
    y=alt.Y('trips:Q', axis=alt.Axis(title='Total Transit Trips')),
    color="mode:N",
    tooltip=['month','mode', 'trips']
).interactive()
chart_ridership.save('chart_ridership.html')

In [6]:
top50_c = top50.merge(covid_l, on="fips", how='left')
top50_c.head()

GEOID10  order                  metro_area   fips       date       county  \
0    10162      1  Bridgeport-Stamford, CT-NY   9001 2020-09-30    Fairfield   
1    11350      2                 Buffalo, NY  36029 2020-09-30         Erie   
2    15670      3            Charlotte, NC-SC  37119 2020-09-30  Mecklenburg   
3    16264      4              Chicago, IL-IN  17031 2020-09-30         Cook   
4    16885      5        Cincinnati, OH-KY-IN  39061 2020-09-30     Hamilton   

            state   cases  deaths  
0     Connecticut   20225  1422.0  
1        New York   11500   625.0  
2  North Carolina   28975   360.0  
3        Illinois  145462  5225.0  
4            Ohio   13426   320.0

In [41]:
select_date = "2020-09"
ridership_agg = ridership[ridership["month"] == select_date]
ridership_agg = ridership_agg.groupby(['GEOID10', 'metro_area'], as_index=False).agg({'trips': 'sum', 'lat': 'min', 'lon': 'min'})
ridership_agg.head()

GEOID10                  metro_area    trips        lat        lon
0    10162  Bridgeport-Stamford, CT-NY   561267  41.235154 -73.309377
1    10972             Brownsville, TX    41808  25.966922 -97.474430
2    11350                 Buffalo, NY  1031785  42.926628 -78.823853
3    11755              Burlington, VT   110494  44.480976 -73.161863
4    13375                  Canton, OH   108702  40.828596 -81.430242

In [8]:
ridership_agg = ridership_agg[['GEOID10','trips']]
top50_c = top50_c.merge(ridership_agg, on="GEOID10", how='left')
top50_c.head()

GEOID10  order                  metro_area   fips       date       county  \
0    10162      1  Bridgeport-Stamford, CT-NY   9001 2020-09-30    Fairfield   
1    11350      2                 Buffalo, NY  36029 2020-09-30         Erie   
2    15670      3            Charlotte, NC-SC  37119 2020-09-30  Mecklenburg   
3    16264      4              Chicago, IL-IN  17031 2020-09-30         Cook   
4    16885      5        Cincinnati, OH-KY-IN  39061 2020-09-30     Hamilton   

            state   cases  deaths     trips  
0     Connecticut   20225  1422.0    561267  
1        New York   11500   625.0   1031785  
2  North Carolina   28975   360.0    750275  
3        Illinois  145462  5225.0  15097626  
4            Ohio   13426   320.0    640183

In [9]:
#Plot relationship between covid cases and transit trips

chart_scatter = alt.Chart(top50_c).mark_circle(size=200).encode(
    x=alt.X('cases', axis=alt.Axis(title='COVID-19 Cases (Cumulative)')),
    y=alt.Y('trips', axis=alt.Axis(title='Total Transit Trips in September 2020')),
    color=alt.Color('state', legend=None),
    tooltip=['metro_area', 'cases', 'deaths', 'trips']
).interactive()
chart_scatter.save('scatter.html')

In [10]:
#Query covid data for a sample city

search_county = "San Francisco"
covid_s = covid[covid["county"] == search_county]
covid_s['new_cases'] = covid_s['cases'].diff()
covid_s['new_deaths'] = covid_s['deaths'].diff()
covid_s['cases_avg'] = covid_s.iloc[:,6].rolling(window=7).mean()
covid_s['deaths_avg'] = covid_s.iloc[:,7].rolling(window=7).mean()
covid_s['DateTime'] = pd.to_datetime(covid_s['date'])
covid_s

date         county       state  fips  cases  deaths  new_cases  \
49     2020-02-02  San Francisco  California  6075      2     0.0        NaN   
57     2020-02-03  San Francisco  California  6075      2     0.0        0.0   
65     2020-02-04  San Francisco  California  6075      2     0.0        0.0   
73     2020-02-05  San Francisco  California  6075      2     0.0        0.0   
82     2020-02-06  San Francisco  California  6075      2     0.0        0.0   
...           ...            ...         ...   ...    ...     ...        ...   
761167 2020-11-26  San Francisco  California  6075  15056   160.0      179.0   
764385 2020-11-27  San Francisco  California  6075  15159   160.0      103.0   
767603 2020-11-28  San Francisco  California  6075  15380   160.0      221.0   
770821 2020-11-29  San Francisco  California  6075  15453   160.0       73.0   
774039 2020-11-30  San Francisco  California  6075  15574   160.0      121.0   

        new_deaths   cases_avg  deaths_avg   DateTime  
49             NaN         NaN         NaN 2020-02-02  
57             0.0         NaN         NaN 2020-02-03  
65             0.0         NaN         NaN 2020-02-04  
73             0.0         NaN         NaN 2020-02-05  
82             0.0         NaN         NaN 2020-02-06  
...            ...         ...         ...        ...  
761167         0.0  142.285714    0.571429 2020-11-26  
764385         0.0  129.285714    0.571429 2020-11-27  
767603         0.0  150.000000    0.285714 2020-11-28  
770821         0.0  143.571429    0.285714 2020-11-29  
774039         0.0  147.000000    0.285714 2020-11-30  

[303 rows x 11 columns]

In [11]:
#Plot covid cases with altair

chart_cases = alt.Chart(covid_s).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month')),
    y=alt.Y('cases_avg:Q', axis=alt.Axis(title='New COVID-19 Cases Reported')),
)
chart_cases.save('chart_cases.html')

In [12]:
#Plot covid deaths with altair

chart_deaths = alt.Chart(covid_s).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month')),
    y=alt.Y('deaths_avg:Q', axis=alt.Axis(title='New COVID-19 Deaths Reported')),
    color=alt.value('orange'),
)
chart_deaths.save('chart_deaths.html')

In [42]:
fig = [px.scatter_mapbox(ridership_agg,
                        lat="lat",
                        lon="lon",
                        size="trips",
                        # https://plotly.com/python/builtin-colorscales/
                        color_continuous_scale=px.colors.sequential.Jet,
                        size_max=80,
                        zoom=3,
                        hover_data=["trips"],
                        hover_name='metro_area'
                        ),
       px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
      ]
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(
    title=f'Total Transit Trips in {select_date}',
)
fig.show()
fig.write_html("map.html")